In [3]:
!pip install dalex

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.0 MB 35.0 MB/s 
  Created wheel for dalex: filename=dalex-1.5.0-py3-none-any.whl size=1043321 sha256=2f006fbe37e4f1be29c47bdcceb885d9f4f06433bfaff084a3fd8969c27885ac
  Stored in directory: /root/.cache/pip/wheels/76/31/8c/c78df586df31b3f3e0c4ecc759ee73e175545cff5548201996
Successfully built dalex


In [128]:
import dalex as dx
import xgboost

import sklearn
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
from copy import copy
import warnings
warnings.filterwarnings("ignore")

import platform
print(f'Python {platform.python_version()}')

Python 3.7.15


# Load/preprocess data

In [142]:
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/archive_4/adult.csv")

def encode_and_bind(original_dataframe, feature_to_encode):
    dummies = pd.get_dummies(original_dataframe[feature_to_encode], drop_first=True)
    res = pd.concat([original_dataframe.drop(feature_to_encode,axis=1), dummies], axis=1)
    return(res)

onehot_df = copy(df)
for feature in ["workclass","education","marital-status","occupation","relationship","gender","race","native-country"]:
  onehot_df = encode_and_bind(onehot_df,feature)
di = {"<=50K": 0, ">50K": 1}
onehot_df = onehot_df.replace({"income":di})
onehot_df.head()

X,Y = onehot_df.drop("income",axis=1),onehot_df.income
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, Y, test_size=0.1, random_state=42)

# Model

In [143]:
model = xgboost.XGBClassifier(
    n_estimators=150, 
    max_depth=4, 
    use_label_encoder=True, 
    eval_metric="logloss",
    
    enable_categorical=True,
    tree_method="hist"
)

model2 = xgboost.XGBClassifier(
    n_estimators=4, 
    max_depth=2, 
    use_label_encoder=True, 
    eval_metric="logloss",
    
    enable_categorical=True,
    tree_method="hist"
)

model.fit(X_train, y_train)
model2.fit(X_train, y_train)

XGBClassifier(enable_categorical=True, eval_metric='logloss', max_depth=2,
              n_estimators=4, tree_method='hist', use_label_encoder=True)

In [177]:
def pf_xgboost_classifier_categorical(model, df):
    df.loc[:, df.dtypes == 'object'] =\
        df.select_dtypes(['object'])\
        .apply(lambda x: x.astype('category'))
    return model.predict_proba(df)[:, 1]

explainer = dx.Explainer(model, X_test, y_test, predict_function=pf_xgboost_classifier_categorical)
explainer.label="big model original"
explainer2 = dx.Explainer(model2, X_test, y_test, predict_function=pf_xgboost_classifier_categorical)
explainer2.label="simpler model"

Preparation of a new explainer is initiated

  -> data              : 4885 rows 100 cols
  -> target variable   : Parameter 'y' was a pandas.Series. Converted to a numpy.ndarray.
  -> target variable   : 4885 values
  -> model_class       : xgboost.sklearn.XGBClassifier (default)
  -> label             : Not specified, model's class short name will be used. (default)
  -> predict function  : <function pf_xgboost_classifier_categorical at 0x7fca292a5680> will be used
  -> predict function  : Accepts only pandas.DataFrame, numpy.ndarray causes problems.
  -> predicted values  : min = 0.000342, mean = 0.243, max = 0.999
  -> model type        : classification will be used (default)
  -> residual function : difference between y and yhat (default)
  -> residuals         : min = -0.991, mean = -0.0026, max = 0.993
  -> model_info        : package xgboost

A new explainer has been created!
Preparation of a new explainer is initiated

  -> data              : 4885 rows 100 cols
  -> target var

In [178]:
pd.concat([explainer.model_performance().result, explainer2.model_performance().result], axis=0)

,recall,precision,f1,accuracy,auc
big model original,0.656997,0.790554,0.717614,0.875947,0.931842
simpler model,0.524744,0.763975,0.622155,0.847083,0.854550


In [179]:
X_test.columns

Index(['age', 'fnlwgt', 'educational-num', 'capital-gain', 'capital-loss',
       'hours-per-week', 'Federal-gov', 'Local-gov', 'Never-worked', 'Private',
       'Self-emp-inc', 'Self-emp-not-inc', 'State-gov', 'Without-pay', '11th',
       '12th', '1st-4th', '5th-6th', '7th-8th', '9th', 'Assoc-acdm',
       'Assoc-voc', 'Bachelors', 'Doctorate', 'HS-grad', 'Masters',
       'Preschool', 'Prof-school', 'Some-college', 'Married-AF-spouse',
       'Married-civ-spouse', 'Married-spouse-absent', 'Never-married',
       'Separated', 'Widowed', 'Adm-clerical', 'Armed-Forces', 'Craft-repair',
       'Exec-managerial', 'Farming-fishing', 'Handlers-cleaners',
       'Machine-op-inspct', 'Other-service', 'Priv-house-serv',
       'Prof-specialty', 'Protective-serv', 'Sales', 'Tech-support',
       'Transport-moving', 'Not-in-family', 'Other-relative', 'Own-child',
       'Unmarried', 'Wife', 'Male', 'Asian-Pac-Islander', 'Black', 'Other',
       'White', 'Cambodia', 'Canada', 'China', 'Columbia'

In [180]:
protected_variable = X_test.Male.apply(lambda x: "male" if x else "non-male")
privileged_group = "male"

fobject = explainer.model_fairness(
    protected=protected_variable,
    privileged=privileged_group
)

In [181]:
fobject.fairness_check()

Bias detected in 2 metrics: FPR, STP

Conclusion: your model is not fair because 2 or more criteria exceeded acceptable limits set by epsilon.

Ratios of metrics, based on 'male'. Parameter 'epsilon' was set to 0.8 and therefore metrics should be within (0.8, 1.25)
               TPR       ACC       PPV      FPR       STP
non-male  0.889387  1.117438  1.081841  0.17284  0.303846


In [182]:
fobject.plot()

The predictive pairty is in very good shape, same for equal opportunity.

Statistical parity is violated: the prediction of high income is far more likely for males. This might be due to the fact that historically (and currently), females are often homemakers, which is bound to lower income

In [183]:
fobject2 = explainer2.model_fairness(
    protected=protected_variable,
    privileged=privileged_group
)
fobject2.fairness_check()

Bias detected in 2 metrics: FPR, STP

Conclusion: your model is not fair because 2 or more criteria exceeded acceptable limits set by epsilon.

Ratios of metrics, based on 'male'. Parameter 'epsilon' was set to 0.8 and therefore metrics should be within (0.8, 1.25)
               TPR       ACC       PPV       FPR       STP
non-male  0.854749  1.152605  1.118194  0.142857  0.285714


In [184]:
fobject.plot(fobject2, show=False).\
    update_layout(autosize=False, width=800, height=450, legend=dict(yanchor="top", y=0.99, xanchor="right", x=0.99))



Let's do reweighting

In [185]:
from dalex.fairness import resample, reweight
from copy import copy

In [186]:
weights = reweight(
    X_train.Male.apply(lambda x: "male" if x else "non-male"), 
    y_train, 
    verbose=False
)
model_reweight = copy(model)
model_reweight.fit(X_train, y_train, sample_weight=weights)
explainer_reweight = dx.Explainer(
    model_reweight, 
    X_test, 
    y_test, 
    label='reweighting',
    verbose=False
)
fobject_reweight = explainer_reweight.model_fairness(
    protected_variable, 
    privileged_group
)


In [189]:
fobject.plot([fobject2,fobject_reweight], show=False).\
    update_layout(autosize=False, width=800, height=450, legend=dict(yanchor="bottom", y=0.99, xanchor="right", x=0.99))



In [190]:
pd.concat([explainer.model_performance().result, explainer2.model_performance().result,explainer_reweight.model_performance().result], axis=0)

,recall,precision,f1,accuracy,auc
big model original,0.656997,0.790554,0.717614,0.875947,0.931842
simpler model,0.524744,0.763975,0.622155,0.847083,0.854550
reweighting,0.623720,0.791983,0.697852,0.870420,0.927071
